In [1]:
import os

import matplotlib.pyplot as plt
from scipy import stats

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 180px
}
</style>

In [4]:
#Variable initialization

current_dist_index = 0

#noise_removal_filter = NoiseRemovalFilter.NoiseRemovalFilter()

#Default values
population_size = 100 # Participant per experiment
initial_budget = 10.
number_experiments = 10 # Times to run the experiment
json_name = 'blood.json'

# Parameters for output graphs
output_folder = 'output/'
output_format = 'eps'
label_font_size=8
xlabel_rotation = 10
ylimit=[0,50]

In [5]:
json_files = filter(lambda x: x.lower().endswith('.json'), [f for f in os.listdir('.')])

file_selection = widgets.Select(
    options=json_files,
    value=json_name,
    disabled=False
)

precision_text = widgets.BoundedIntText(value=2,
    disabled=False,
    step = 1,
    min=0)

epsilon_text = widgets.BoundedFloatText(
    value=initial_budget,
    disabled=False,
    readout=True,
    readout_format='.'+str(precision_text.value)+'f',
    step = 10**-precision_text.value,
    min = 10**-precision_text.value
)

number_experiments_text = widgets.BoundedIntText(
    value=number_experiments,
    disabled=False,
    step = 1,
    min=1)

output_folder_text = widgets.Text(
    value=output_folder,
    placeholder='Folder for output',
    disabled=False
)

output_format_selection = widgets.Select(
    options=['eps', 'png', 'pdf', 'svg'],
    value=output_format,
    disabled=False
)

load_button = widgets.Button(
    description='Load JSON',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='hourglass-start'
)

dist_accordion = widgets.Accordion()
dist_accordion.layout.padding = '0% 0% 5% 1%'
dist_accordion.layout.width = '99%'

json_box = widgets.VBox([widgets.Label('Input JSON:'),file_selection, load_button])
json_box.layout.align_items='center'
exp_box = widgets.VBox([widgets.Label('Number of Experiments'),number_experiments_text])
exp_box.layout.align_items='center'
epsilon_box = widgets.VBox([widgets.Label('Total Privacy Budget:'), epsilon_text])
epsilon_box.layout.align_items='center'

params_box = widgets.HBox([json_box, exp_box, epsilon_box])
params_box.layout.align_items='flex-start'

precison_box = widgets.VBox([widgets.Label('Precision:'), precision_text])
precison_box.layout.align_items='center'
folder_box = widgets.VBox([widgets.Label('Output Folder:'), output_folder_text])
folder_box.layout.align_items='center'
format_box = widgets.VBox([widgets.Label('Output Format:'), output_format_selection])
format_box.layout.align_items='center'

output_box = widgets.HBox([precison_box, folder_box, format_box])
output_box.layout.align_items='flex-start'

start_tab = widgets.Tab()
tab_contents = ['Experiment Parameters', 'Output Options']
children = [params_box, output_box]
start_tab.children = children
for i in range(len(tab_contents)):
    start_tab.set_title(i, tab_contents[i])
    
run_button = widgets.Button(
    description='Run Experiments',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check'
)

run_box = widgets.VBox([run_button])
run_box.layout.width='99%'
run_box.layout.align_items='center'


number_html = widgets.HTML()

toggle_button = widgets.ToggleButton(
    description='Switch Mode',
    disabled=False,
    value=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='edit'
)   

toggle_box = widgets.VBox([toggle_button])
toggle_box.layout.width = '99%'
toggle_box.layout.align_items = 'center'

In [6]:
# GUI elements
uniform_buttons = []
draw_buttons = []
shape_sliders = []

sliders_precision = 4

dists = ['Custom', 'Geom', 'Logser', 'Planck', 'Uniform', 'Zipf']
dist_maps = []
dropdowns = []

sliders_matrix = []
custom_sliders = []
texts = []

dist_boxes = []

out = widgets.Output()
out.layout.align_items = 'center'
fig = None

In [7]:
#### GUI dist elements
uniform_button = widgets.Button(description='Uniform', tooltip='Set all sliders to a uniform distribution')
normal_button = widgets.ToggleButton(
    value=False,
    description='Normal',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Approximate a normal distribution',
    icon='times'
)
draw_button = widgets.Button(description='Draw', icon='fa-bar-chart', button_style='success')
shape_slider = widgets.FloatSlider(min=0., max=5., step=0.1, value=0., continuous_update=False, description='Shape', disabled=True) 
shape_slider.layout.visibility='hidden'

sliders_precision = 4

dists = ['Custom', 'Geom', 'Logser', 'Planck', 'Uniform', 'Zipf']
distMap = {'Geom' : lambda amount_answers, shape: stats.geom.pmf(list(np.arange(start=1., stop=amount_answers+1, step=1.)), shape), #Shape: 'p'
           'Logser' : lambda amount_answers, shape: stats.logser.pmf(list(np.arange(start=1., stop=amount_answers+1, step=1.)), shape), #Shape: 'p'
           'Planck' : lambda amount_answers, shape: stats.planck.pmf(list(np.arange(start=0., stop=amount_answers, step=1.)), shape), #Shape: 'lambda'
           'Uniform' : lambda amount_answers, _: stats.randint.pmf(list(np.arange(start=0., stop=amount_answers, step=1.)), 0, amount_answers),
          'Zipf' : lambda amount_answers, shape: stats.zipf.pmf(list(np.arange(start=1., stop=amount_answers+1, step=1.)), shape)} #Shape: 'a'
dropdown = widgets.Dropdown(options=dists, value='Custom', description='Distribution:', disabled=False)

sliders = []
custom_sliders = []
texts = []

box = widgets.VBox()

out = widgets.Output()
out.layout.align_items = 'center'
fig = None

In [8]:
def toggle_labels(change):
    
    if(normal_button.value):
        for text in texts:
            normal_button.button_style = 'success'
            normal_button.icon = 'check'
            text.disabled = False
            text.layout.visibility = 'visible'
    
    else:
        for text in texts:
            normal_button.button_style = 'danger'
            normal_button.icon = 'times'
            text.disabled = True
            text.layout.visibility = 'hidden'
        
def draw_graph(change):
    # Normalize the slider values
    s = sorted_sliders()
    probabilities = pd.Series([slider.value for slider in s])
    normalize_by = probabilities.sum()
    
    # Remove old GUI and show new
    clear_output()
    display(box, out)
    
    # Plot new graph
    plot(pd.Series([str(slider.description) for slider in s]), (probabilities/normalize_by))
    
def slide_action(change):

    repaint_layout()
    # Set labels visible/invisible
    toggle_labels(None)

    # Calculate slider values unless 'custom' is chosen
    if(dropdown.value != 'Custom'):
        update_dist(None)
    
    draw_graph(None)
    
def plot(x_axis, y_axis):
    with sns.axes_style("darkgrid"):
        fig.clear()
        ax = fig.add_subplot(1, 1, 1)
        ax.set_xlabel('Answer')
        ax.set_ylabel('')
        ax.set_ylim((0,1.))
        sns.barplot(x=x_axis, y=y_axis, ax=ax, palette='pastel')
        fig.canvas.draw_idle()
    with out:
        clear_output()
        display(fig)
        
def sorted_sliders():
    # Sort according to value
    s = list(sliders)
    
    # Weights set, use that order, otherwise alphabetical
    if [t.value in range(1,len(sliders)) for t in texts]:
        zippy = zip([slider for slider in sliders], [text.value for text in texts])
        zippy.sort(key= lambda (slider,weight): weight)
        s = [slider for (slider,weight) in zippy]
    else:    
        s.sort(key=lambda x: x.description, reverse=False)
        
    s.sort(key=lambda x: x.value, reverse=True)
    return s

In [9]:
def draw(change):
    # Layout update
    repaint_layout()
    # Set labels visible/invisible
    toggle_labels(None)

    # Calculate slider values unless 'custom' is chosen
    if(dropdown.value != 'Custom'):
        update_dist(None)
    
    draw_graph(None)
    
def repaint_layout():
    global box, texts
    
    # Add a label to each slider element
    pair_boxes = []
    
    for i in range(1, len(sliders)+1):
        texts[i-1].layout.width='150px'
        vbox = widgets.VBox([sliders[i-1], texts[i-1]])
        vbox.layout.align_items='flex-end'
        pair_boxes.append(vbox)

     # Put the layout together
    slider_box = widgets.HBox(sliders)
    dist_box = widgets.HBox([dropdown])
    box = widgets.VBox([dist_box, widgets.HBox(pair_boxes), shape_slider, draw_button])
    box.layout.align_items='center'